# Create Folders

In [12]:
import os
os.makedirs('configs', exist_ok=True)
os.makedirs('train', exist_ok=True)
os.makedirs('inference', exist_ok=True)

In [5]:
!pip install wandb

In [6]:
import os
from kaggle_secrets import UserSecretsClient

# Load W&B key from Kaggle Secrets
os.environ["WANDB_API_KEY"] = UserSecretsClient().get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "llm-from-scratch"
os.environ["WANDB_ENTITY"] = "rahulkrish28-california-state-university-fullerton"

In [7]:
%%writefile configs/train_base.yaml

model_name: gpt2
dataset_name: wikitext
dataset_config: wikitext-2-raw-v1

max_length: 512
batch_size: 2
num_epochs: 1

learning_rate: 5e-5
weight_decay: 0.01
warmup_ratio: 0.1
max_grad_norm: 1.0

output_dir: outputs/train_lm
seed: 42

Writing configs/train_base.yaml


In [8]:
%%writefile train/train_lm.py

import math
import yaml
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler
)
from datasets import load_dataset
from tqdm import tqdm

# -------------------------
# Load config
# -------------------------
with open("configs/train_base.yaml") as f:
    cfg = yaml.safe_load(f)

torch.manual_seed(cfg["seed"])

device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_name"])
model.to(device)

# -------------------------
# Dataset
# -------------------------
dataset = load_dataset(
    cfg["dataset_name"],
    cfg["dataset_config"]
)

def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=cfg["max_length"],
        padding=False,
    )

    # FILTER EMPTY SEQUENCES
    input_ids = []
    for ids in tokens["input_ids"]:
        if len(ids) > 0:
            input_ids.append(ids)

    return {"input_ids": input_ids}

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

train_loader = DataLoader(
    tokenized["train"],
    batch_size=cfg["batch_size"],
    shuffle=True,
    collate_fn=data_collator
)

# -------------------------
# Optimizer
# -------------------------
# ---- Defensive casting ----
cfg["learning_rate"] = float(cfg["learning_rate"])
cfg["weight_decay"] = float(cfg["weight_decay"])

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=cfg["learning_rate"],
    weight_decay=cfg["weight_decay"]
)

num_training_steps = cfg["num_epochs"] * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

def sanity_check(batch):
    assert batch["input_ids"].dim() == 2
    assert batch["input_ids"].size(1) > 0
    
# -------------------------
# Training Loop
# -------------------------
model.train()
progress = tqdm(range(num_training_steps))

for epoch in range(cfg["num_epochs"]):
    for batch in train_loader:
        sanity_check(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress.update(1)
        progress.set_postfix(loss=loss.item())

# -------------------------
# Perplexity
# -------------------------
ppl = math.exp(loss.item())
print(f"Final Perplexity: {ppl:.2f}")

model.save_pretrained(cfg["output_dir"])
tokenizer.save_pretrained(cfg["output_dir"])

Writing train/train_lm.py


In [9]:
!python train/train_lm.py

2025-12-21 23:11:45.242640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766358705.429914     113 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766358705.482356     113 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766358705.928892     113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766358705.928939     113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766358705.928943     113 computation_placer.cc:177] computation placer alr

In [10]:
%%writefile configs/inference.yaml

# =========================
# Inference Configuration
# =========================

# -------------------------
# Model
# -------------------------
model_dir: "outputs/train_lm"   # same as cfg["output_dir"] during training
device: "auto"                  # auto | cuda | cpu

# -------------------------
# Tokenization
# -------------------------
max_input_length: 512           # truncate prompt if longer than this

# -------------------------
# Generation Parameters
# -------------------------
max_new_tokens: 150             # number of tokens to generate
do_sample: true                 # sampling vs greedy decoding

temperature: 0.8                # randomness (1.0 = neutral)
top_p: 0.9                      # nucleus sampling
top_k: 50                       # optional, can be null
repetition_penalty: 1.0         # >1.0 reduces repetition

# -------------------------
# Special Tokens
# -------------------------
pad_token: "eos"                # use EOS as PAD
skip_special_tokens: true

# -------------------------
# Runtime
# -------------------------
use_fp16: false                 # set true if GPU supports fp16
batch_size: 1                   # for batch inference

Writing configs/inference.yaml


In [13]:
%%writefile inference/inference.py

import yaml
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# -------------------------
# Load Inference Config
# -------------------------
with open("configs/inference.yaml") as f:
    cfg = yaml.safe_load(f)

# -------------------------
# Device
# -------------------------
if cfg["device"] == "auto":
    device = "cuda" if torch.cuda.is_available() else "cpu"
else:
    device = cfg["device"]

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_dir"])

# Pad token handling
if cfg["pad_token"] == "eos":
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_dir"])

if cfg.get("use_fp16", False) and device == "cuda":
    model = model.half()

model.to(device)
model.eval()

# -------------------------
# Generation Function
# -------------------------
@torch.no_grad()
def generate(prompt: str):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=cfg["max_input_length"],
    ).to(device)

    generation_kwargs = {
        "max_new_tokens": cfg["max_new_tokens"],
        "do_sample": cfg["do_sample"],
        "temperature": cfg["temperature"],
        "top_p": cfg["top_p"],
        "pad_token_id": tokenizer.eos_token_id,
        "repetition_penalty": cfg["repetition_penalty"],
    }

    # Optional top-k
    if cfg.get("top_k") is not None:
        generation_kwargs["top_k"] = cfg["top_k"]

    outputs = model.generate(**inputs, **generation_kwargs)

    return tokenizer.decode(
        outputs[0],
        skip_special_tokens=cfg["skip_special_tokens"]
    )

# -------------------------
# Example Run
# -------------------------
if __name__ == "__main__":
    prompt = "Once upon a time in a futuristic city,"
    text = generate(prompt)
    print(text)

Writing inference/inference.py


In [14]:
!python inference/inference.py

2025-12-21 23:40:41.062552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766360441.081778     197 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766360441.087250     197 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766360441.104857     197 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766360441.104885     197 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766360441.104890     197 computation_placer.cc:177] computation placer alr

In [6]:
%%writefile train/train_lm_wandb.py

import math
import yaml
import torch
import wandb
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler
)
from datasets import load_dataset
from tqdm import tqdm

# -------------------------
# Load config
# -------------------------
with open("configs/train_base.yaml") as f:
    cfg = yaml.safe_load(f)

wandb.init(
    project="llm-from-scratch",
    config=cfg
)

torch.manual_seed(cfg["seed"])
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_name"])
model.to(device)

# -------------------------
# Dataset
# -------------------------
dataset = load_dataset(
    cfg["dataset_name"],
    cfg["dataset_config"]
)

def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=cfg["max_length"],
        padding=False,
    )

    # FILTER EMPTY SEQUENCES
    input_ids = []
    for ids in tokens["input_ids"]:
        if len(ids) > 0:
            input_ids.append(ids)

    return {"input_ids": input_ids}

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

train_loader = DataLoader(
    tokenized["train"],
    batch_size=cfg["batch_size"],
    shuffle=True,
    collate_fn=data_collator
)

# -------------------------
# Optimizer & Scheduler
# -------------------------
cfg["learning_rate"] = float(cfg["learning_rate"])
cfg["weight_decay"] = float(cfg["weight_decay"])
cfg["warmup_ratio"] = float(cfg.get("warmup_ratio", 0.0))
cfg["max_grad_norm"] = float(cfg.get("max_grad_norm", 1.0))

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=cfg["learning_rate"],
    weight_decay=cfg["weight_decay"]
)

num_training_steps = cfg["num_epochs"] * len(train_loader)
num_warmup_steps = int(cfg["warmup_ratio"] * num_training_steps)

lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

def sanity_check(batch):
    assert batch["input_ids"].dim() == 2
    assert batch["input_ids"].size(1) > 0
    
# -------------------------
# Training Loop
# -------------------------
model.train()
global_step = 0

progress = tqdm(range(num_training_steps))

for epoch in range(cfg["num_epochs"]):
    for batch in train_loader:
        sanity_check(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(),
            cfg["max_grad_norm"]
        )

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        lr = lr_scheduler.get_last_lr()[0]

        wandb.log({
            "train/loss": loss.item(),
            "train/perplexity": math.exp(loss.item()),
            "train/lr": lr,
            "train/grad_norm": grad_norm,
            "train/step": global_step
        })

        global_step += 1
        progress.update(1)
        progress.set_postfix(loss=loss.item(), lr=lr)

# -------------------------
# Save
# -------------------------
model.save_pretrained(cfg["output_dir"])
tokenizer.save_pretrained(cfg["output_dir"])

wandb.finish()
torch.cuda.max_memory_allocated()

Writing train/train_lm_wandb.py


In [ ]:
!python train/train_lm_wandb.py

In [7]:
%%writefile train/train_lm_ddp.py

import os
import math
import yaml
import torch
import torch.distributed as dist
import wandb

from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler
)
from datasets import load_dataset
from tqdm import tqdm

# -------------------------
# DDP setup
# -------------------------
def setup_ddp():
    dist.init_process_group(backend="gloo")
    rank = dist.get_rank()
    world_size = dist.get_world_size()
    return rank, world_size

def cleanup_ddp():
    dist.destroy_process_group()

rank, world_size = setup_ddp()

# -------------------------
# Load config
# -------------------------
with open("configs/train_base.yaml") as f:
    cfg = yaml.safe_load(f)

torch.manual_seed(cfg["seed"])

device = torch.device("cpu")  # CPU-safe DDP

# -------------------------
# W&B init (ONLY rank 0)
# -------------------------
if rank == 0:
    wandb.init(
        project="llm-from-scratch",
        name=cfg.get("wandb_run_name", "ddp-cpu-debug"),
        config=cfg
    )

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_name"])
model.to(device)

model = DDP(model)

# -------------------------
# Dataset
# -------------------------
dataset = load_dataset(
    cfg["dataset_name"],
    cfg["dataset_config"]
)

def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=cfg["max_length"],
        padding=False,
    )

    input_ids = [ids for ids in tokens["input_ids"] if len(ids) > 0]
    return {"input_ids": input_ids}

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

sampler = DistributedSampler(
    tokenized["train"],
    num_replicas=world_size,
    rank=rank,
    shuffle=True
)

train_loader = DataLoader(
    tokenized["train"],
    batch_size=cfg["batch_size"],
    sampler=sampler,
    collate_fn=data_collator
)

# -------------------------
# Optimizer & Scheduler
# -------------------------
cfg["learning_rate"] = float(cfg["learning_rate"])
cfg["weight_decay"] = float(cfg["weight_decay"])

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=cfg["learning_rate"],
    weight_decay=cfg["weight_decay"]
)

num_training_steps = cfg["num_epochs"] * len(train_loader)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# -------------------------
# Training Loop
# -------------------------
def sanity_check(batch):
    assert batch["input_ids"].dim() == 2
    assert batch["input_ids"].size(1) > 0

model.train()
global_step = 0

for epoch in range(cfg["num_epochs"]):
    sampler.set_epoch(epoch)

    for batch in tqdm(train_loader, disable=(rank != 0)):
        sanity_check(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        if rank == 0:
            wandb.log({
                "train/loss": loss.item(),
                "train/lr": lr_scheduler.get_last_lr()[0],
                "epoch": epoch,
                "step": global_step
            })

        global_step += 1

    if rank == 0:
        print(f"Epoch {epoch} Loss: {loss.item():.4f}")

# -------------------------
# Save only on rank 0
# -------------------------
if rank == 0:
    model.module.save_pretrained(cfg["output_dir"])
    tokenizer.save_pretrained(cfg["output_dir"])
    wandb.finish()

cleanup_ddp()

Writing train/train_lm_ddp.py


In [ ]:
!torchrun --nproc_per_node=2 train/train_lm_ddp.py

W1221 21:17:17.453000 101 torch/distributed/run.py:774] 
W1221 21:17:17.453000 101 torch/distributed/run.py:774] *****************************************
W1221 21:17:17.453000 101 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1221 21:17:17.453000 101 torch/distributed/run.py:774] *****************************************
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` 

In [8]:
%%writefile train/train_lm_amp_ckpt.py

import math
import yaml
import torch
import wandb

from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler
)
from datasets import load_dataset
from tqdm import tqdm

# -------------------------
# Load config
# -------------------------
with open("configs/train_base.yaml") as f:
    cfg = yaml.safe_load(f)

torch.manual_seed(cfg["seed"])
device = torch.device("cuda")

# -------------------------
# Initialize W&B
# -------------------------
wandb.init(
    project="llm-from-scratch",
    config=cfg,
    name="train_amp_ckpt"
)

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_name"])
model.gradient_checkpointing_enable()  # 🔥 Gradient checkpointing
model.to(device)

# -------------------------
# Dataset
# -------------------------
dataset = load_dataset(
    cfg["dataset_name"],
    cfg["dataset_config"]
)

def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=cfg["max_length"],
        padding=False,
    )
    input_ids = [ids for ids in tokens["input_ids"] if len(ids) > 0]
    return {"input_ids": input_ids}

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

train_loader = DataLoader(
    tokenized["train"],
    batch_size=cfg["batch_size"],
    shuffle=True,
    collate_fn=data_collator
)

# -------------------------
# Optimizer & Scheduler
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=float(cfg["learning_rate"]),
    weight_decay=float(cfg["weight_decay"])
)

num_training_steps = cfg["num_epochs"] * len(train_loader)
lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

scaler = GradScaler()  # 🔥 AMP scaler

# -------------------------
# Training Loop
# -------------------------
def sanity_check(batch):
    assert batch["input_ids"].dim() == 2
    assert batch["input_ids"].size(1) > 0

model.train()
global_step = 0

for epoch in range(cfg["num_epochs"]):
    for batch in tqdm(train_loader):
        sanity_check(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        with autocast():  # 🔥 AMP
            outputs = model(**batch)
            loss = outputs.loss

        # Backprop with AMP
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        lr_scheduler.step()

        # Log metrics to W&B
        wandb.log({
            "train/loss": loss.item(),
            "train/perplexity": math.exp(loss.item()),
            "train/lr": lr_scheduler.get_last_lr()[0],
            "train/global_step": global_step
        })
        global_step += 1

    # Epoch-level logging
    print(f"Epoch {epoch} Loss: {loss.item():.4f} | Perplexity: {math.exp(loss.item()):.2f}")
    wandb.log({
        "epoch/loss": loss.item(),
        "epoch/perplexity": math.exp(loss.item()),
        "epoch": epoch
    })

# -------------------------
# Save
# -------------------------
model.save_pretrained(cfg["output_dir"])
tokenizer.save_pretrained(cfg["output_dir"])

wandb.finish()
torch.cuda.max_memory_allocated()

Writing train/train_lm_amp_ckpt.py


In [ ]:
%%writefile train/train_lm_lora.py

import time
import math
import yaml
import torch
import wandb
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler
)

from datasets import load_dataset
from tqdm import tqdm

# PEFT LoRA
from peft import LoraConfig, get_peft_model, TaskType

# -------------------------
# Load config
# -------------------------
with open("configs/train_base.yaml") as f:
    cfg = yaml.safe_load(f)

torch.manual_seed(cfg["seed"])
device = torch.device("cuda")

# -------------------------
# W&B init
# -------------------------
wandb.init(project="llm-from-scratch", config=cfg, name="LoRA-Training")

# -------------------------
# Tokenizer & Model
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(cfg["model_name"])
model.gradient_checkpointing_enable()  # gradient checkpointing
model.to(device)

# -------------------------
# Freeze base model & Add LoRA
# -------------------------
for param in model.parameters():
    param.requires_grad = False

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # check trainable params

# -------------------------
# Dataset
# -------------------------
dataset = load_dataset(
    cfg["dataset_name"],
    cfg["dataset_config"]
)

def tokenize_fn(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=cfg["max_length"],
        padding=False,
    )
    input_ids = [ids for ids in tokens["input_ids"] if len(ids) > 0]
    return {"input_ids": input_ids}

tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

train_loader = DataLoader(
    tokenized["train"],
    batch_size=cfg["batch_size"],
    shuffle=True,
    collate_fn=data_collator
)

# -------------------------
# Optimizer & Scheduler
# -------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=float(cfg["learning_rate"]),
    weight_decay=float(cfg["weight_decay"])
)

num_training_steps = cfg["num_epochs"] * len(train_loader)
lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)

scaler = GradScaler()  # AMP

# -------------------------
# Training Loop + W&B logging
# -------------------------
def sanity_check(batch):
    assert batch["input_ids"].dim() == 2
    assert batch["input_ids"].size(1) > 0
    
model.train()
global_step = 0
step_times = []

for epoch in range(cfg["num_epochs"]):
    for batch in tqdm(train_loader):
        sanity_check(batch)
        batch = {k: v.to(device) for k, v in batch.items()}

        start = time.time()
        with autocast():
            outputs = model(**batch)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        lr_scheduler.step()
        end = time.time()

        step_time = end - start
        step_times.append(step_time)

        # Log GPU memory
        max_mem = torch.cuda.max_memory_allocated() / 1024**2  # MB
        lr = lr_scheduler.get_last_lr()[0]

        # W&B logging
        wandb.log({
            "train/loss": loss.item(),
            "train/perplexity": math.exp(loss.item()),
            "train/lr": lr,
            "train/max_memory_MB": max_mem,
            "train/step_time_s": step_time,
            "train/global_step": global_step
        })

        global_step += 1

        tqdm.write(
            f"Epoch {epoch} | Step {global_step} | Loss: {loss.item():.4f} | "
            f"Mem: {max_mem:.1f}MB | Step Time: {step_time:.2f}s"
        )

# -------------------------
# Save LoRA adapters only
# -------------------------
model.save_pretrained(cfg["output_dir"] + "_lora")
tokenizer.save_pretrained(cfg["output_dir"])

wandb.finish()